# Implementation of A* search algorithm on grid

## Grid

To approximate the shortest path in real-life situations, like in maps, games where there can be many hindrances.

We can consider a 2D Grid having several obstacles and we start from a source cell (coloured red below) to reach towards a goal cell (coloured green below)

<p align="center">
	<img src="https://cdncontribute.geeksforgeeks.org/wp-content/uploads/a_-search-algorithm-1.png" width="450">
</p>
_Image source: https://cdncontribute.geeksforgeeks.org_


In [147]:
class SquareGrid:
    """
    SquareGrid class represents agent's environment
    """
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        
    def get_map(self, input_map):
        for i in range(self.height):
            for j in range(self.width):
                if input_map[i * self.width + j] == '#':
                    self.grid[i][j] = 1
    
    def in_bounds(self, i, j):
        return 0 <= j < self.width and 0 <= i < self.height
    
    def traversable(self, i, j):
        if self.grid[i][j] == 0:
            return True
        return False
    
    def neighbors(self, i, j, diagonal=False, cutcorners=False, squeeze=False):
        """
        function, that returns neighbor nodes to the current (i, j) according to the following paramenters:
        diagonal: True, if diagonal moves are allowed
        cutcorners: True, if your agent is allowed to cut corners
        squeeze: True, if agent is also allowed to squeeze between obstacles
        """
        neighbors = []
        for di in range(-1, 2):
            for dj in range(-1, 2):
                if (di != 0 or dj != 0) and \
                    self.in_bounds(i + di, j + dj) and \
                    self.traversable(i + di, j + dj):
                    if di != 0 and dj != 0:
                        if not diagonal: continue
                        elif not cutcorners:
                            if not self.traversable(i, j + dj) or \
                               not self.traversable(i + di, j): continue
                        elif not squeeze:
                            if not self.traversable(i, j + dj) and \
                               not self.traversable(i + di, j): continue
                    neighbors.append((i + di, j + dj))
        return neighbors

In [148]:
import numpy as np

input_map = '''
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
'''
# for more maps look into maps_examples)

g = SquareGrid(30, 15)
g.get_map(input_map.translate({ ord(c): None for c in ' \n\t\r' })) #reduce all unnecessary whitespaces 
for gr in g.grid:
    print(*gr)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


## A* (A star)

A* Search algorithm is one of the best and popular technique used in path-finding and graph traversals.

A* is like [Dijkstra’s Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) in that it can be used to find a shortest path. A* is like [Greedy Best-First-Search](https://en.wikipedia.org/wiki/Best-first_search) in that it can use a _heuristic_ to guide itself.

<img src="https://cdn-images-1.medium.com/max/2000/1*sbUuCeAb1EHgYBszGOXSWg.gif" width="650"> 

#### First let's define our search node class, which will contain all the information about the search node:

- it's g and f values, 
- pointer to the node's parent.

In [149]:
import math

class Node:
    """
    Node class represents a search node
    i, j: coordinates of the node on a square grid
    g: it's g-values
    h: the values of heuristic function
    parent: pointer on the parent element 
    """
    def __init__(self, i, j, g=math.inf, h=math.inf, parent=None):
        self.i = i
        self.j = j
        self.g = g
        self.F = self.g + h
        self.parent = parent

#### Now let's define a class for our OPEN list
We should be able to quickly exctract node with the minimum $F$ value and search for the node with the same $i$ and $j$ coordinates.

In [168]:
class OpenList:
    """
    OpenList class represents a struct for efficient OPEN list representation
    """
    def __init__(self, height):
        self.elements = [[] for _ in range(height)]
        self.size = 0
    
    def empty(self):
        for elem in self.elements:
            if len(elem) != 0:
                return False
        return True
    
    def get(self):
        best_F = math.inf
        best_coord = 0
        for coord in range(len(self.elements)):
            if len(self.elements[coord]) == 0:
                continue
            if self.elements[coord][0].F < best_F:
                best_coord = coord
                best_F = self.elements[coord][0].F
                
        # after we found the element with the lowest F value, we need to delete found element from OPEN list
        best = self.elements[best_coord].pop(0)
        # and return it
        return best
    
    def put(self, item):
        if len(self.elements[item.i]) == 0:
            self.elements[item.i].append(item)
            self.size += 1
            return
        position = 0
        position_found = False
        
        #find the position on which to insert our new element
        # meanwhile you should also look for the node with the same coordinates in the OPEN already
        
        for i in range(len(self.elements[item.i])):
            if not position_found and self.elements[item.i][i].F >= item.F:
                position = i
                position_found = True
                    
            if self.elements[item.i][i].j == item.j:
                if item.F >= self.elements[item.i][i].F:
                    return
                elif position == i:
                    self.elements[item.i][i].F = item.F
                    self.elements[item.i][i].g = item.g
                    self.elements[item.i][i].parent = item.parent
                    return
                else:
                    self.elements[item.i].pop(position)
                    self.size -= 1
                    break
                    
        self.size += 1           
        self.elements[item.i].insert(position, item)
        return

### Heuristics for grid maps

#### Manhattan distance

   The heuristic on a square grid where you can move in 4 directions should be $\alpha$ times the Manhattan distance:

In [169]:
def manhattan_heuristic(a_i, a_j, b_i, b_j, alpha=1):
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return alpha * (dx + dy)

#### Diagonal distance

   If your map allows diagonal movement you need a different heuristic. Here we compute the number of steps you take if you can’t take a diagonal, then subtract the steps you save by using the diagonal:

In [170]:
def diagonal_heuristic(a_i, a_j, b_i, b_j, alpha=1, alpha_2=1):
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return alpha * (dx + dy) + (alpha_2 - 2 * alpha) * min(dx, dy)

When $\alpha = 1$ and $\alpha_2 = 1$, this is called the *Chebyshev distance*. When $\alpha = 1$ and $\alpha_2 = \sqrt{2}$, this is called the *octile distance*.

#### Euclidean distance

If your units can move at any angle (instead of grid directions), then you should probably use a straight line distance:

In [171]:
def euclidean_heuristic(a_i, a_j, b_i, b_j, alpha=1):
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return alpha * math.sqrt(dx * dx + dy * dy)

There is much more...

__Let's start with A*__

In [172]:
def calculate_heuristic(a_i, a_j, goal_i, goal_j, heuristic_type='euclid', alpha=1):
    if heuristic_type == 'euclidean':
        return euclidean_heuristic(a_i, a_j, goal_i, goal_j, alpha=alpha)
    if heuristic_type == 'octile':
        return diagonal_heuristic(a_i, a_j, goal_i, goal_j, alpha=alpha, alpha_2=math.sqrt(2))
    if heuristic_type == 'chebyshev':
        return diagonal_heuristic(a_i, a_j, goal_i, goal_j, alpha=alpha) 
    if heuristic_type == 'manhattan':
        return manhattan_heuristic(a_i, a_j, goal_i, goal_j, alpha=alpha) 
    
def calculate_cost(a_i, a_j, b_i, b_j):
    return math.sqrt(abs(a_i - b_i) * abs(a_i - b_i) + abs(a_j - b_j) * abs(a_j - b_j))


def search(grid, start_i, start_j, goal_i, goal_j,
           heuristic_type='euclidean', 
           diagonal=False, 
           cutcorners=False, 
           squeeze=False):
    
    OPEN = OpenList(grid.height)
    start_node = Node(start_i, start_j, 0, 
                      calculate_heuristic(start_i, start_j, goal_i, goal_j, heuristic_type))
    OPEN.put(start_node)
    CLOSE = dict()
    
    while not OPEN.empty():
        current = OPEN.get()
        CLOSE[current.i * grid.width + current.j] = current
        
        if current.i == goal_i and current.j == goal_j:
            print("Path has been found!")
            return current, CLOSE
        for (i, j) in grid.neighbors(current.i, current.j):
            if i * grid.width + j not in CLOSE:
                new_node = Node(i, j, current.g + calculate_cost(current.i, current.j, i, j), 
                                calculate_heuristic(i, j, goal_i, goal_j, heuristic_type),
                                current)
                OPEN.put(new_node)
    
    return current, CLOSE

In [174]:
%%time

g = SquareGrid(30, 15)
g.get_map(input_map.translate({ ord(c):None for c in ' \n\t\r' }))
goal, CLOSE = search(g, 0, 0, 14, 29)

Path has been found!
CPU times: user 9.74 ms, sys: 800 µs, total: 10.5 ms
Wall time: 10.6 ms


In [178]:
print("Found length of the path:", goal.g)

Found length of the path: 43.0


In [179]:
def make_path(goal):
    current = goal
    path = []
    while current.parent:
        path.append((current.i, current.j))
        current = current.parent;
    path.append((current.i, current.j))
    return path[::-1]

path = make_path(goal)
print("Found path:", *path)

Found path: (0, 0) (0, 1) (0, 2) (0, 3) (0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (0, 10) (0, 11) (0, 12) (0, 13) (0, 14) (0, 15) (0, 16) (1, 16) (1, 17) (2, 17) (2, 18) (3, 18) (3, 19) (4, 19) (4, 20) (5, 20) (6, 20) (7, 20) (7, 21) (7, 22) (7, 23) (8, 23) (8, 24) (9, 24) (9, 25) (10, 25) (10, 26) (11, 26) (11, 27) (12, 27) (12, 28) (13, 28) (13, 29) (14, 29)


In [180]:
def print_path(g, path):
    new_grid = g.grid.copy()
    for i in range(g.height):
        for j in range(g.width):
            if (i, j) in path:
                new_grid[i][j] = '*'
            else:
                new_grid[i][j] = g.grid[i][j]
    for gr in new_grid:
        print(*gr)


path = make_path(goal)
print_path(g, path)

* * * * * * * * * * * * * * * * * 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 * * 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 * * 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 * * 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 * * 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 * 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 * 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 * * * * 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 * * 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 * * 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 * * 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 * * 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 * * 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 * *
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 *


### Further work

Feel free to experiment, calculating distances and execution time according to different allowed movements or different heuristics